<a href="https://colab.research.google.com/github/AkshathaBolla/ANN/blob/main/Practice_QuantumNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module I: Practice Exercise - 'Quantum-Net' (Logic Edition)

### **Context: The Qubit Control System**
You are tuning a quantum computer. You have sensor data from 500 qubits.
**Inputs:**
1.  **Temperature (mK)** (milliKelvin)
2.  **Coherence Time (ns)**
3.  **Gate Error Rate (%)**

**Goals (Swapped from previous):**
* **Level 1 (Regression):** Predict the **Processing Speed (MHz)** based on stability metrics.
* **Level 2 (Classification):** Predict if the state is **'Entangled' (1)** or **'Decohered' (0)**.

---
**INSTRUCTIONS:**
This is a **Logic-Heavy Practice**.
* **Level 2** requires implementing **Early Stopping** logic manually.
* **Level 4** requires updating both **Weight** and **Bias** manually.

In [7]:
# CELL 1: DATA GENERATION (Run this first)
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

torch.manual_seed(2025)

# 500 Qubits
N = 500
# Features: Temp(10-100), Time(100-5000), Error(0-1)
X = torch.rand(N, 3) * torch.tensor([90, 4900, 1]) + torch.tensor([10, 100, 0])

# Target A: Processing Speed (Regression)
# Speed = Coherence/10 - Temp*2
y_speed = (X[:, 1] / 10) - (X[:, 0] * 2) + torch.randn(N)*10
y_speed = y_speed.view(-1, 1).float()

# Target B: Entanglement State (Classification)
# Entangled if Error < 0.3 AND Coherence > 2000
state_score = X[:, 1] - (X[:, 2] * 5000)
y_state = (state_score > 1000).float().view(-1, 1)

print(f"Quantum Data Ready. X: {X.shape}, y_speed: {y_speed.shape}, y_state: {y_state.shape}")

Quantum Data Ready. X: torch.Size([500, 3]), y_speed: torch.Size([500, 1]), y_state: torch.Size([500, 1])


## **Level 1: The Speed Predictor (Regression)**
**Objective:** Build a regression model to predict Processing Speed.

**The Blueprint:**
1.  **Architecture:** `Linear(3, 64) -> ReLU -> Linear(64, 1)`.
    * *Critial:* Regression models do NOT use Sigmoid/Softmax at the end.
2.  **Training:**
    * Loss: `MSELoss`
    * Optimizer: `Adam` (lr=0.01)
    * Epochs: 100

In [8]:
# LEVEL 1: WRITE YOUR CODE HERE

# 1. Define 'SpeedModel'
class SpeedModel(nn.Module):
  def __init__(self):
    super(SpeedModel,self).__init__()

    self.linear=nn.Linear(3,64)
    self.relu=nn.ReLU()
    self.linear1=nn.Linear(64,1)

    # TODO: Linear -> ReLU -> Linear
  def forward(self,x):
      x=self.linear(x)
      x=self.relu(x)
      x=self.linear1(x)
      return x

model_reg = SpeedModel()

# 2. Optimizer & Loss
criterion=nn.MSELoss()
optimizer=optim.Adam(model_reg.parameters(),lr=0.01)

# 3. Training Loop (No DataLoaders yet)
for epoch in range(100):
  optimizer.zero_grad()
  outputs=model_reg(X)
  loss=criterion(outputs,y_speed)
  loss.backward()
  optimizer.step()


In [9]:
# TEST LEVEL 1
try:
    # Check Architecture
    l2 = list(model_reg.children())[-1]
    assert l2.out_features == 1, "Output layer must have 1 neuron"
    # Check for activation on output (should be none)
    assert not isinstance(list(model_reg.children())[-1], nn.Sigmoid), "Regression should not use Sigmoid on output"

    print("✅ Level 1 Passed: Regression Architecture correct.")
except Exception as e:
    print(f"❌ Level 1 Fail: {e}")

✅ Level 1 Passed: Regression Architecture correct.


## **Level 2: The State Classifier (Early Stopping)**
**Objective:** Predict 'Entangled' (1) vs 'Decohered' (0) using **Early Stopping**.

**Step 2.1: Data Handling**
* Split data: **Train (400)** and **Validation (100)**.
* Create DataLoaders (Batch Size = 32).

**Step 2.2: The Logic**
* Model: `Linear(3, 128) -> ReLU -> Linear(128, 1) -> Sigmoid`.
* **Task:** Write a training loop that monitors `val_loss`.
* **Condition:** If `val_loss` does not improve for **5 consecutive epochs** (patience=5), stop training and print "Early Stopping Triggered".

*Hint: Keep track of `best_loss` and a counter variable.*

In [10]:
# LEVEL 2: WRITE YOUR CODE HERE

# 1. Split & DataLoaders
X_train,X_val=X[:400],X[400:]
y_train,y_val=y_state[:400],y_state[400:]

train_dataset=TensorDataset(X_train,y_train)
val_dataset=TensorDataset(X_val,y_val)

train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True)
val_loader=DataLoader(val_dataset,batch_size=32)
# 2. Define Model
class StateClassifier(nn.Module):
  def __init__(self):
    super(StateClassifier,self).__init__()
    self.layer1=nn.Linear(3,128)
    self.relu=nn.ReLU()
    self.layer2=nn.Linear(128,1)
    self.sigmoid=nn.Sigmoid()
  def forward(self,x):
    x=self.layer1(x)
    x=self.relu(x)
    x=self.layer2(x)
    x=self.sigmoid(x)
    return x
model_cls = StateClassifier()
criterion=nn.BCELoss()
optimizer=optim.Adam(model_cls.parameters(),lr=0.01)

# 3. Training Loop with Early Stopping
best_loss = float('inf')
patience = 5
trigger_times = 0

for epoch in range(200):
    # Train...
    model_cls.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model_cls(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    # Validate...
    model_cls.eval()
    val_loss=0.0

    with torch.no_grad():
        for inputs,targets in val_loader:
           outputs=model_cls(inputs)
           loss=criterion(outputs,targets)
           val_loss+=loss.item()

    val_loss/=len(val_loader)

    # Check Early Stopping Logic...
    if val_loss < best_loss:
       best_loss=val_loss
       trigger_times=0
    else:
       trigger_times+=1


    if trigger_times >= patience:
      print("Early Stopping Triggered")
      break

Early Stopping Triggered


In [11]:
# TEST LEVEL 2
try:
    assert 'trigger_times' in locals() or 'patience' in locals(), "You must define patience or a counter for early stopping"
    assert isinstance(list(model_cls.modules())[-1], nn.Sigmoid), "Classification must end with Sigmoid"
    print("✅ Level 2 Passed: Logic checks out.")
except Exception as e:
    print(f"❌ Level 2 Fail: {e}")

✅ Level 2 Passed: Logic checks out.


## **Level 3: The Lab (Manual Loss & Activations)**

**Part 3.1: Manual Huber Loss**
Implement `my_huber_loss(pred, target, delta=1.0)`.
* If $|error| < delta$: Loss = $0.5 * error^2$
* If $|error| >= delta$: Loss = $delta * (|error| - 0.5 * delta)$
* *Note: This is robust to outliers.*

**Part 3.2: The Tournament**
Compare these configs on the **Regression Task**:
1.  **"Baseline"**: `ReLU` + `MSELoss`
2.  **"Smooth"**: `ELU` + `L1Loss` (MAE)
3.  **"Robust"**: `LeakyReLU` + `my_huber_loss`

In [12]:
# LEVEL 3.1: MANUAL HUBER LOSS
def my_huber_loss(pred, target, delta=1.0):
    # TODO: Implement formula above
    error=pred-target
    abs_error=torch.abs(error)

    quadratic =torch.minimum(abs_error,torch.tensor(delta))
    linear=abs_error-quadratic

    loss=0.5*quadratic**2+delta*linear
    return loss.mean()


# LEVEL 3.2: CONFIG
experiments = [
    {"name": "Baseline", "act": nn.ReLU(), "loss": nn.MSELoss()}, # ReLU, MSE
    {"name": "Smooth",   "act": nn.ELU(), "loss": nn.L1Loss()}, # ELU, L1
    {"name": "Robust",   "act": nn.LeakyReLU(), "loss": my_huber_loss}  # LeakyReLU, my_huber_loss
]

In [13]:
# TEST LEVEL 3
try:
    # Test Huber
    p = torch.tensor([10.0]); t = torch.tensor([0.0]) # Error = 10
    # Since 10 > 1.0:
    # Loss = 1.0 * (10 - 0.5*1.0) = 9.5
    assert abs(my_huber_loss(p, t).item() - 9.5) < 0.1, "Math Fail: Huber Loss incorrect"

    assert isinstance(experiments[1]['act'], nn.ELU), "Exp 2 must be ELU"
    print("✅ Level 3 Passed: Manual Loss and Config correct.")
except Exception as e:
    print(f"❌ Level 3 Fail: {e}")

✅ Level 3 Passed: Manual Loss and Config correct.


## **Level 4: The Mechanic (Bias Update)**
**Objective:** Manual Gradient Descent including **Bias**.

**Goal:** Learn $y = w \cdot x + b$

1.  **Values:** $x=2.0$, $y=10.0$
2.  **Weights:** $w=3.0$, $b=1.0$
3.  **Loss:** Squared Error $L = (pred - y)^2$
4.  **Task:** Calculate gradients for $w$ **AND** $b$, then update both.
    * $\frac{dL}{dw} = 2(pred - y) \cdot x$
    * $\frac{dL}{db} = 2(pred - y) \cdot 1$  <-- Note the difference!

In [14]:
# LEVEL 4: WRITE YOUR CODE HERE
lr = 0.1
x = 2.0
y = 10.0
w = 3.0
b = 1.0

# 1. Forward (Pred)
pred = w*x+b

# 2. Gradients (dw, db)
error=pred-y
grad_w = 2*error*x
grad_b = 2*error*1

# 3. Updates
w_new = w-lr*grad_w
b_new = b-lr*grad_b

In [15]:
# TEST LEVEL 4
try:
    # Pred = 3*2 + 1 = 7
    # Error = 7-10 = -3
    # grad_w = 2 * -3 * 2 = -12
    # grad_b = 2 * -3 * 1 = -6
    # w_new = 3.0 - (0.1 * -12) = 4.2
    # b_new = 1.0 - (0.1 * -6) = 1.6

    assert abs(w_new - 4.2) < 0.01, f"Weight Fail: Expected 4.2, got {w_new}"
    assert abs(b_new - 1.6) < 0.01, f"Bias Fail: Expected 1.6, got {b_new}"
    print("✅ Level 4 Passed: Multi-parameter Gradient Descent successful!")
except Exception as e:
    print(f"❌ Level 4 Fail: {e}")

✅ Level 4 Passed: Multi-parameter Gradient Descent successful!
